# HFA Regressions
Created: 04/08/24 \
Updated: 06/14/2024


In [ ]:
import numpy as np
import mne
from glob import glob
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
from scipy.stats import zscore, linregress, ttest_ind, ttest_rel, ttest_1samp, pearsonr, spearmanr
import pandas as pd
from mne.preprocessing.bads import _find_outliers
import os 
import joblib
import re
import datetime
import scipy
import random
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.regression.mixed_linear_model import MixedLM 
from joblib import Parallel, delayed
import pickle
import itertools
import time 
from matplotlib.ticker import StrMethodFormatter



import warnings
warnings.filterwarnings('ignore')




In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Specify root directory for un-archived data and results 
base_dir = '/sc/arion/projects/guLab/Alie/SWB/'
anat_dir = f'{base_dir}ephys_analysis/recon_labels/'
neural_dir = f'{base_dir}ephys_analysis/data/'
behav_dir = f'{base_dir}swb_behav_models/data/behavior_preprocessed/'
save_dir = f'{base_dir}ephys_analysis/results/'
script_dir = '/hpc/users/finka03/swb_ephys_analysis/scripts/'

date = datetime.date.today().strftime('%m%d%Y')
print(date)

In [ ]:
import sys
sys.path.append(f'{base_dir}ephys_analysis/LFPAnalysis/')

from LFPAnalysis import lfp_preprocess_utils, sync_utils, analysis_utils, nlx_utils

sys.path.append(f'{script_dir}analysis_notebooks/')

from ieeg_tools import *


In [ ]:
subj_ids = list(pd.read_excel(f'{base_dir}ephys_analysis/subj_info/SWB_subjects.xlsx', sheet_name='Usable_Subjects', usecols=[0]).PatientID)
n_subj = len(subj_ids)
subj_ids


In [ ]:
bdi_list = pd.read_excel(f'{base_dir}ephys_analysis/subj_info/SWB_subjects.xlsx', sheet_name='Usable_Subjects', usecols=[3])
bdi_list = list(bdi_list.SWB_BDI)
subj_info_df = pd.DataFrame({'subj_id':subj_ids,'bdi':bdi_list})
subj_info_df

# Load data
- First run - load all power_epochs data, after just save as pickle dict
- Get subj metadata for regression dfs + compute logRT & logRT_t1

In [ ]:
# #### Try Beta Power 
# all_subj_hfa_power = {}
# all_subj_metadata   = {}

# hfa_freqs = [70,200]

# # load pow epochs data 
# for subj_id in subj_ids: 
#     # load power epochs for single subject 
#     power_epochs = mne.time_frequency.read_tfrs(f'{neural_dir}{subj_id}/{subj_id}_CpeOnset-tfr.h5')[0]
#     power_epochs = power_epochs.crop(tmin=0,tmax=2.0,include_tmax=True)
#     all_subj_metadata[subj_id] = power_epochs.metadata.copy()
    
#     # extract power df with mean beta power for each epoch for each elec 
#     power_df = power_epochs.to_data_frame()
#     del power_epochs 
#     hfa_power_df = power_df[(power_df.freq >= hfa_freqs[0])&(power_df.freq <= hfa_freqs[1])].groupby(
#         ['epoch']).agg('mean').reset_index()
#     del power_df 
#     all_subj_hfa_power[subj_id] = hfa_power_df
#     del hfa_power_df
    
    
# # create a binary pickle file 
# pickle.dump(all_subj_hfa_power,open(f'{save_dir}all_subj_hfa_power.pkl',"wb"))
# # pickle.dump(all_subj_metadata,open(f'{save_dir}all_subj_metadata.pkl',"wb"))

    

In [ ]:
# paths = glob(f'{save_dir}all_subj_hfa_power_*.pkl')
# load_date = str(np.max([int(path.split('_')[-1][:-4]) # find most recent saved df to load 
#                         for path in paths]))

all_subj_hfa_power = pickle.load(open(f'{save_dir}all_subj_hfa_power.pkl','rb'))
all_subj_metadata = pickle.load(open(f'{save_dir}regression_data/all_subj_metadata.pkl','rb'))

# all_subj_hfa_power

In [ ]:
### update metadata - zscore continuous regressors + add RT info

for subj_id in subj_ids:
    subj_df         = all_subj_metadata[subj_id]
    rounds_t1       = subj_df.Round.tolist()[1:]
    logRT_list      = np.log(subj_df['RT']).tolist() # take log of RT
    logRT_t1_list   = logRT_list[1:] # get logRT_t1
    GambleChoice_t1 = subj_df.GambleChoice.tolist()[1:]
    GambleEV_t1     = subj_df.GambleEV.tolist()[1:]
    TrialType_t1    = subj_df.TrialType.tolist()[1:]
    TrialEV         = subj_df.GambleEV - subj_df.SafeBet
    TrialEV_t1      = TrialEV[1:].tolist()
    rounds_t1.append(np.nan)
    logRT_t1_list.append(np.nan) # add nan to logRT_t1 (no t1 for last round)
    GambleChoice_t1.append(np.nan)
    GambleEV_t1.append(np.nan)
    TrialType_t1.append(np.nan) 
    TrialEV_t1.append(np.nan)
    subj_df['logRT']           = logRT_list
    subj_df['logRT_t1']        = logRT_t1_list
    subj_df['Round_t1']        = rounds_t1
    subj_df['GambleChoice_t1'] = GambleChoice_t1
    subj_df['GambleEV_t1']     = GambleEV_t1
    subj_df['TrialType_t1']    = TrialType_t1
    subj_df['TrialEV']         = TrialEV
    subj_df['TrialEV_t1']      = TrialEV_t1
    subj_df['bdi']             = subj_info_df.bdi[subj_info_df.subj_id==subj_id].values[0]
    subj_df['subj_id']         = subj_id
    
    # update metadata
    all_subj_metadata[subj_id] = subj_df
    
    
pickle.dump(all_subj_metadata,open(f'{save_dir}regression_data/all_subj_metadata_clean.pkl',"wb"))
    


In [ ]:
roi_reref_labels_master_df = pd.read_csv(glob(f'{base_dir}ephys_analysis/results/roi_info/roi_reref_labels_master.csv')[0])
roi_reref_labels_master_df = roi_reref_labels_master_df.drop(columns=['Unnamed: 0'])


#### add roi info to master df 
hfa_master_df = []


# variables to zscore after data cleaning - 
continuous_regressors = ['GambleEV','GambleEV_t1','SafeBet','Profit','TotalProfit','RPE',
                         'decisionCPE', 'decisionRegret','decisionRelief','TrialEV','TrialEV_t1']

for subj_id,pow_df in zip(all_subj_hfa_power.keys(),all_subj_hfa_power.values()):
    
     # reshape pow_df 
        pow_reg_df = pow_df.melt(id_vars=['epoch','freq'],
                                    value_vars=pow_df.columns.drop(['epoch','time','freq']).tolist().copy(),
                                    var_name='reref_ch_names', value_name='band_pow',
                                    ignore_index = False)

        pow_reg_df['subj_id'] = subj_id # add subj_id to df values
        # reshape metadata df 
        metadata_df = all_subj_metadata[subj_id].reset_index()
        metadata_df = metadata_df.rename(columns={'index':'epoch'})

        # merge pow and metadata dfs keeping epoch as index 
        subj_regression_df = pd.merge(pow_reg_df, metadata_df,left_on=['epoch','subj_id'],
                                    right_on=['epoch','subj_id']).reset_index(drop=True).set_index('epoch')
        
        subj_regression_df['epoch_num'] = subj_regression_df.index # keep epoch as a column

        # add roi info 
        subj_epochs = pow_reg_df.epoch.unique()
        # extract roi_reref_labels_df for each subj 
        subj_roi_reref_labels = roi_reref_labels_master_df[roi_reref_labels_master_df['subj_id']==subj_id]
        # add epochs to df - repeat each elec id for every epoch to join with master df 
        subj_roi_reref_labels = roi_reref_labels_master_df[roi_reref_labels_master_df['subj_id']==subj_id]
        subj_roi_reref_labels = (pd.concat([subj_roi_reref_labels]*len(subj_epochs),
                                keys = subj_epochs,
                                names = ['epoch',None])).reset_index(level=1,drop=True) #make sure epochs are the index
        subj_roi_reref_labels['epoch_num'] = subj_roi_reref_labels.index

        # create unique reref ch id for regression
        subj_roi_reref_labels['unique_reref_ch'] = subj_roi_reref_labels[['subj_id', 'reref_ch_names']].agg('_'.join, axis=1)

        # merge regression df with roi reref info df  
        subj_regression_df = pd.merge(subj_regression_df, subj_roi_reref_labels,left_on=['subj_id',
                                    'reref_ch_names','epoch_num'],right_on=['subj_id',
                                    'reref_ch_names','epoch_num']).reset_index(drop=True).set_index('epoch_num')
        
        subj_regression_df['epoch'] = subj_regression_df.index
        
        ### cleaning within subject BEFORE zscoring regressors! 
        
        # remove trials where RT is too fast (random choice)
        subj_regression_df = subj_regression_df[subj_regression_df.RT>0.3]
        # remove trials where there is no choice at trial t
        subj_regression_df = subj_regression_df[(subj_regression_df.GambleChoice=='gamble') | 
                                                (subj_regression_df.GambleChoice == 'safe')].reset_index(drop=True)
        # remove trials where there is no choice for trial t1 (RT_t1 is nonsense)
        subj_regression_df = subj_regression_df[(subj_regression_df.GambleChoice_t1=='gamble') | 
                                                (subj_regression_df.GambleChoice_t1 == 'safe')].reset_index(drop=True)
        # remove trials where there is no outcome
        subj_regression_df = subj_regression_df[(subj_regression_df.Outcome=='good') | 
                                                (subj_regression_df.Outcome == 'bad')].reset_index(drop=True)
        # remove epoch 76 bc photodiode times are not correct at breakpoint 
        subj_regression_df = subj_regression_df[subj_regression_df.Round != 76]
        # remove trials where logRT_t1 is inf or -inf (means RT = 0)
        subj_regression_df = subj_regression_df[~np.isinf(subj_regression_df.logRT_t1)]
        # format GambleChoice + TrialType vars as categories 
        subj_regression_df['GambleChoice']    = subj_regression_df['GambleChoice'].astype('category')
        subj_regression_df['GambleChoice_t1'] = subj_regression_df['GambleChoice_t1'].astype('category')
        subj_regression_df['TrialType']       = subj_regression_df['TrialType'].astype('category')
        subj_regression_df['TrialType_t1']    = subj_regression_df['TrialType_t1'].astype('category')
        # make sure Round_t1 is an integer
        subj_regression_df['Round_t1']        = subj_regression_df['Round_t1'].astype('Int64')
        
        # zscore continuous regressors AFTER removing bad trials!! 
        for reg in continuous_regressors:
            subj_regression_df[reg] = norm_zscore(subj_regression_df[reg].values) # zscore is /2std

        hfa_master_df.append(subj_regression_df)


    
    
hfa_master_df = pd.concat(hfa_master_df)   
# reorder columns 
hfa_master_df = hfa_master_df[['subj_id','bdi','epoch','freq','reref_ch_names','band_pow','unique_reref_ch',
                                 'roi','hemi','ch_label4roi','ch_type4roi','loc4roi','Round', 'Round_t1',
                                 'logRT_t1','logRT','RT','TrialType','TrialType_t1','GambleChoice',
                                 'GambleChoice_t1','GambleEV', 'GambleEV_t1','SafeBet','TrialEV','TrialEV_t1',
                                 'Profit','TotalProfit','RPE','decisionCPE', 'decisionRegret','decisionRelief']]



In [ ]:
# hfa_master_df.to_csv(f'{save_dir}hfa_master_df.csv')
hfa_master_df

In [ ]:
roi_reref_labels_master_df.roi.value_counts()

In [ ]:
print(hfa_master_df.band_pow.max(),hfa_master_df.band_pow.mean(),hfa_master_df.band_pow.std(),hfa_master_df.band_pow.min())


In [ ]:
hfa_reg_df = hfa_master_df.copy()
hfa_reg_df.roi.unique().tolist()

# Mixed effects with just CPE 


In [ ]:
cpe_hfa_model = smf.mixedlm(
    formula='band_pow ~ 1 + decisionCPE',
    re_formula = ' decisionCPE',
    data = hfa_reg_df,groups=hfa_reg_df['unique_reref_ch'], missing='drop')
cpe_hfa_model_fit = cpe_hfa_model.fit()
print(cpe_hfa_model_fit.summary())

# Feature Selection

In [ ]:
vif_scores(hfa_reg_df,['GambleChoice','TrialType','TotalProfit','TrialEV','decisionCPE']) #,'logRT_t1'


In [ ]:
######## make all possible variable combinations for regressions 
main_var  = 'decisionCPE'
iter_vars = ['C(GambleChoice)','C(TrialType)','TotalProfit','TrialEV']


In [ ]:
var_combos = [[v,main_var] for v in iter_vars]

for i in range(1,len(iter_vars)-1):
    iter_list = [list(tup)+[main_var] for tup in list(itertools.combinations(iter_vars,i+1))]
    var_combos.extend(iter_list)


var_combos = [combo for combo in var_combos if ('C(GambleChoice)' in combo and 'C(TrialType)' not in combo)|
              ('C(GambleChoice)' not in combo and 'C(TrialType)' in combo)|
              ('C(GambleChoice)'not in combo and 'C(TrialType)' not in combo)]



var_combos = [combo for combo in var_combos if ('C(GambleChoice)' in combo and 'TrialEV' not in combo)|
              ('C(GambleChoice)' not in combo and 'TrialEV' in combo)|('C(GambleChoice)'not in combo 
                                                                       and 'TrialEV' not in combo)]


var_combos = [combo for combo in var_combos if ('TrialEV' in combo and 'C(TrialType)' not in combo)|
              ('TrialEV' not in combo and 'C(TrialType)' in combo)|('TrialEV'not in combo 
                                                                    and 'C(TrialType)' not in combo)]
var_combos

In [ ]:
# import time 
start = time.time() # start timer

all_models = {}
no_conv = []

for test_vars in var_combos:
    model_key = ''.join(test_vars[:-1]) ### make string of covariate var names to use as dict key
    model_fit = fit_mixed_model(hfa_reg_df,test_vars,outcome_var,rand_eff_var,reml=False)
    # check if model converged
    if model_fit.converged:
        print(model_key,' converged!')
        all_models[model_key] = {'var_list':test_vars,
                                 'fe_coeff':model_fit.params.to_numpy()[1:len(model_fit.fe_params)],
                                 'fe_pvals':model_fit.pvalues.to_numpy()[1:len(model_fit.fe_params)],
                                 'rsq':compute_marginal_rsq(model_fit),
                                 'bic':model_fit.bic}
    else:
        print(model_key,' did not converge :(')
        no_conv.append(model_key)
    del model_fit

end = time.time()
print('{:.4f} s'.format(end-start)) # print time elapsed for computation (approx 20 seconds per channel)    

In [ ]:
print(no_conv)

In [ ]:
all_models

In [ ]:
model_summary = {key:all_models[key]['bic'] for key in all_models.keys()}
model_summary

In [ ]:
print(list(model_summary.keys())[np.where(list(model_summary.values())==np.min(list(model_summary.values())))[0][0]])


In [ ]:
bic_list = [model_summary[key] for key in model_summary.keys()]
bic_list.sort()

ordered_list = [list(model_summary.keys())[np.where(list(model_summary.values()) == bic)[0][0]] for bic in bic_list]
ordered_list


In [ ]:
[(key,all_models[key]['fe_pvals'][-1]) for key in ordered_list]  


In [ ]:
print(list(model_summary.keys())[np.where(list(model_summary.values())==np.min(list(model_summary.values())))[0][0]])
print(np.min(list(model_summary.values())))

In [ ]:
list(all_models[list(model_summary.keys())[np.where(list(model_summary.values())==np.min(list(model_summary.values())))[0][0]]]['fe_pvals'])


In [ ]:
rsq_summary = {key:all_models[key]['rsq'] for key in all_models.keys()}
rsq_summary

In [ ]:
sorted_rsq = list(rsq_summary.values()).copy()
sorted_rsq.sort(reverse=True)
sorted_rsq

In [ ]:
sorted_rsq_models = {f'{list(rsq_summary.keys())[np.where(list(rsq_summary.values()) == rsq)[0][0]]}':rsq for rsq in sorted_rsq}
sorted_rsq_models

In [ ]:
list(zip(ordered_list,list(sorted_rsq_models.keys())))


# Model Evaluation
once you've selected the best ~3 models you need to evaluate the model output 
1. evaluate the model using cpd and/or generalized liklihood ratio test"
2. check fstats
3. tstat betas from zero 
4. visualization

In [ ]:
outcome_var  = 'band_pow'
rand_eff_var = 'unique_reref_ch'

In [ ]:
final_vars = all_models[list(model_summary.keys())[np.where(list(model_summary.values())==np.min(list(model_summary.values())))[0][0]]]['var_list']

final_vars

In [ ]:
final_vars = ['C(GambleChoice)', 'TotalProfit', 'decisionCPE']

In [ ]:
###### BEST MODEL 

final_fit = fit_mixed_model(hfa_reg_df,final_vars,outcome_var,rand_eff_var,reml=False)
print(final_fit.summary())

In [ ]:
final_fit.bic

In [ ]:
mixed_effects_ftest_ttest(final_fit)

In [ ]:
fe_param_names = list(final_fit.fe_params.index)

In [ ]:
plot_data = pd.concat([pd.DataFrame(
    {**{'subj_id':elec_id.split('_')[0],'reref_ch_names':elec_id.split('_')[1]},'unique_reref_ch':elec_id,
     **{f'raw_{param}':final_fit.random_effects[elec_id][param] for param in list(
            final_fit.random_effects[elec_id].index)}},index=[range(len(fe_param_names))])
        
    for elec_id in final_fit.random_effects.keys()]).reset_index(drop=True)

plot_data['roi'] = [hfa_reg_df.roi[hfa_reg_df.unique_reref_ch == row.unique_reref_ch].values[0]
                                for idx,row in plot_data.iterrows()]

plot_data['bdi'] = [subj_info_df.bdi[subj_info_df.subj_id == subj_id].values[0]
                                for subj_id in plot_data.subj_id]

In [ ]:
plot_data

In [ ]:
#### Get all results for Figs!!!!

fe_intercept   = final_fit.params['Intercept']
fe_cpe_beta    = final_fit.params['decisionCPE']
fe_cpe_beta


#### add fixed effects data to all_elec_plot_data
plot_data['elec_b0']   = np.add(plot_data.raw_Group,fe_intercept)
plot_data['elec_bCPE'] = np.add(plot_data.raw_decisionCPE,fe_cpe_beta)

plot_data = plot_data[~plot_data.roi.isnull()]

In [ ]:
raw_roi_stats = {f'{roi}':ttest_1samp(plot_data.raw_decisionCPE[plot_data.roi == roi].unique(),
                 popmean=0) for roi in plot_data.roi.unique().tolist()}
raw_roi_stats

In [ ]:
roi_stats = {f'{roi}':ttest_1samp(plot_data.elec_bCPE[plot_data.roi == roi].unique(),
                 popmean=0) for roi in plot_data.roi.unique().tolist()}
roi_stats

In [ ]:
bonferr_alpha = 0.05/len(roi_order)
bonferr_alpha

In [ ]:
[(key,raw_roi_stats[key].pvalue) for key in raw_roi_stats.keys() if raw_roi_stats[key].pvalue < bonferr_alpha]

In [ ]:
roi_order = ['dlpfc','vlpfc','dmpfc','ofc','ains','pins','acc','hpc','amy']

roi_plot_data = plot_data.copy()
roi_plot_data = roi_plot_data[roi_plot_data.roi.isin(roi_order)]

# roi_plot_names = {'thalamus':'THAL','hpc':'HPC','amy':'AMY','acc':'ACC','ofc':'OFC','ains':'aINS',
#                   'pins':'pINS','dmpfc':'dmPFC','dlpfc':'dlPFC','vlpfc':'vlPFC'}


In [ ]:
from matplotlib.ticker import StrMethodFormatter

sns.set_context("poster",rc={'axes.linewidth': 1.5})
fig,ax = plt.subplots(figsize=(12, 5),dpi=500)

fig_save_dir = f'{base_dir}ephys_analysis/figs/hfa_regressions/'
os.makedirs(fig_save_dir,exist_ok=True)

sns.barplot(data = roi_plot_data,
            x = 'roi',

            y = 'raw_decisionCPE',
#             y='elec_bCPE',
            color = '#78496b',#78496b 997690
            saturation=.8,
            order=roi_order,
            errorbar=None) 


sns.stripplot(data = roi_plot_data,
            x = 'roi',
            y = 'raw_decisionCPE',
#               y='elec_bCPE',
              color = '#561C46',
              alpha=.3,
              dodge=True,
              order=roi_order,
              size = 8)

ax.axhline(y=0,color='#561C46',linewidth=1.5)
ax.set(xlabel=None)
plt.tick_params(length=0)
plt.ylabel('HFA'+r'$_{{{zpow}}}$'+' cpe encoding')
# plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.3f}')) 
# plt.locator_params(axis='y', nbins=6)

sns.despine()
    
plt.savefig(f'{fig_save_dir}HFA_CPE_barplot_{date}.pdf', format='pdf', metadata=None,
bbox_inches=None, pad_inches=0.1,dpi=500,
facecolor='auto', edgecolor='auto',
backend=None)

In [ ]:
rho_data = []
sig_rho = []

# BDI CORRELATIONS
for roi in plot_data.roi.unique().tolist()[:-1]:
    roi_temp_data = plot_data[plot_data.roi==roi]
    rho,rho_p     = pearsonr(roi_temp_data.bdi,roi_temp_data.raw_decisionCPE)
    rho_data.append(pd.DataFrame({'roi':roi,'rho':rho,'rho_p':rho_p},index=[0]))
    if rho_p < bonferr_alpha: 
        sig_rho.append((roi,rho,rho_p))
    print(f'{roi} pearsonr',rho,rho_p)
rho_data = pd.concat(rho_data).reset_index(drop=True)

In [ ]:
sig_rho

In [ ]:
rho_data['plot_p'] = [0.001 if roi.rho_p < 0.001 else np.round(roi.rho_p,4) for ix,roi in rho_data.iterrows()]

In [ ]:
new_plot_ps = []
for p in rho_data['plot_p']:
    new_p = 0
    
    for ix,val in enumerate(str(p)):
        if val == '0':
            continue
        elif val == '.':
            continue
        else: 
#             print(ix,val)
            new_p = str(p)[:ix+1]
            break
            
            

    new_plot_ps.append(float(new_p))
#         print(val)
rho_data['round_p']   = new_plot_ps
rho_data['round_rho'] = np.round(rho_data.rho,2)
rho_data

In [ ]:

for roi in plot_data.roi.unique().tolist()[:-1]:
    roi_temp_data = plot_data[plot_data.roi==roi]

    rho   = rho_data[rho_data.roi == roi].round_rho.values[0]
    rho_p = rho_data[rho_data.roi == roi].round_p.values[0]
    
        
    fig = plt.figure(figsize=(7,5),dpi=300)

    sns.regplot(data=roi_temp_data,x='bdi',y='raw_decisionCPE',scatter=False,
                fit_reg=True,color = '#561C46') #034975
    
    sns.scatterplot(data = roi_temp_data.groupby(['subj_id']).agg('mean'),x='bdi',
                    y='raw_decisionCPE',color = '#561C46',edgecolor='#561C46',alpha=0.8)
    
#     plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.3f}')) 
    plt.xlabel('BDI-II')
    
    plt.tick_params(length=0) #,width=1.5)
    plt.ylabel('beta'+r'$_{{{zpow}}}$'+' cpe encoding')

    plt.text(np.max(roi_temp_data.groupby(['subj_id']).agg('mean').bdi)-8,
             np.round(np.max(roi_temp_data.groupby(['subj_id']).agg('mean').raw_decisionCPE),4)-0.0005,
             r'$rho$ = '+f'{rho} '+'\n '+r'$p < $'+f'{rho_p}',
             color='#561C46',fontsize=16)

#     plt.locator_params(axis='y', nbins=4)


# #     plt.text(30,(np.round(max(roi_temp_data.groupby(['subj_id']).agg('mean').elec_bCPE),4)),
#     plt.text(30,float(str(max(roi_temp_data.groupby(['subj_id']).agg('mean').elec_bCPE))[:4]),
#              r'$rho$ = '+f'{np.round(rho,2)} '+'\n '+r'$p < $'+f'{np.round(rho_p,4)}',
#              color='#00467e',fontsize=16)

    plt.title(f'{roi}',fontsize=24)

    
    sns.despine()
    
    plt.savefig(f'{fig_save_dir}/{roi}_{rho}_{rho_p}_roi_bdi_HFA-cpe-slope.pdf', format='pdf', metadata=None,
        bbox_inches='tight', pad_inches=0.1,dpi=300,
        facecolor='auto', edgecolor='auto',
        backend=None)
    

# BDI Plots

In [ ]:
bdi_vars    = ['decisionCPE','bdi','decisionCPE:bdi']
outcome_var = 'band_pow'
rand_eff_var = 'unique_reref_ch'
bdi_fit = fit_mixed_model(hfa_reg_df,bdi_vars,outcome_var,rand_eff_var,reml=False)
print(bdi_fit.summary())
    

In [ ]:
fe_param_names = list(bdi_fit.fe_params.index[1:])
fe_param_names

In [ ]:
bdi_plot_data = pd.concat([pd.DataFrame(
    {**{'subj_id':elec_id.split('_')[0],'reref_ch_names':elec_id.split('_')[1]},'unique_reref_ch':elec_id,
     **{f'raw_{param}':bdi_fit.random_effects[elec_id][param] for param in list(
            bdi_fit.random_effects[elec_id].index)}},index=[range(len(fe_param_names))])
        
    for elec_id in bdi_fit.random_effects.keys()]).reset_index(drop=True)

bdi_plot_data['roi'] = [hfa_reg_df.roi[hfa_reg_df.unique_reref_ch == row.unique_reref_ch].values[0]
                                for idx,row in bdi_plot_data.iterrows()]

bdi_plot_data['bdi'] = [subj_info_df.bdi[subj_info_df.subj_id == subj_id].values[0]
                                for subj_id in bdi_plot_data.subj_id]

In [ ]:
bdi_plot_data

In [ ]:
#### Get all results for Figs!!!!

fe_intercept   = bdi_fit.params['Intercept']
fe_cpe_beta    = bdi_fit.params['decisionCPE:bdi']
fe_cpe_beta


In [ ]:
#### add fixed effects data to all_elec_plot_data
bdi_plot_data['elec_b0']   = np.add(bdi_plot_data.raw_Group,fe_intercept)
bdi_plot_data['elec_bCPE'] = np.add(bdi_plot_data['raw_decisionCPE:bdi'],fe_cpe_beta)
bdi_plot_data = bdi_plot_data[~bdi_plot_data.roi.isnull()]

In [ ]:
raw_roi_stats = {f'{roi}':ttest_1samp(bdi_plot_data['raw_decisionCPE:bdi'][bdi_plot_data.roi == roi].unique(),
                 popmean=0) for roi in bdi_plot_data.roi.unique().tolist()}
raw_roi_stats

In [ ]:
roi_stats = {f'{roi}':ttest_1samp(bdi_plot_data.elec_bCPE[bdi_plot_data.roi == roi].unique(),
                 popmean=0) for roi in bdi_plot_data.roi.unique().tolist()}
roi_stats

In [ ]:
# bonferr_alpha = 0.05/len(roi_order)
bonferr_alpha = 0.05/len(bdi_plot_data.roi.unique().tolist())
bonferr_alpha

In [ ]:
print('sig ROIs after multiple comp correction: ',[(roi,raw_roi_stats[roi].pvalue) for roi in raw_roi_stats.keys()
                                                   if raw_roi_stats[roi].pvalue< bonferr_alpha])
# print('sig ROI pvals: ',roi_stats.roi_cpe_pval[roi_beta_results.roi_cpe_pval<=bonferr_alpha].unique().tolist())


In [ ]:
print('sig ROIs after multiple comp correction: ',[(roi,roi_stats[roi].pvalue) for roi in roi_stats.keys()
                                                   if roi_stats[roi].pvalue< bonferr_alpha])
# print('sig ROI pvals: ',roi_stats.roi_cpe_pval[roi_beta_results.roi_cpe_pval<=bonferr_alpha].unique().tolist())




In [ ]:
roi_order = ['dlpfc','vlpfc','dmpfc','ofc','ains','pins','acc','hpc','amy']

In [ ]:
roi_bdi_plot_data = bdi_plot_data.copy()
roi_bdi_plot_data = roi_bdi_plot_data[roi_bdi_plot_data.roi.isin(roi_order)]

# roi_plot_names = {'thalamus':'THAL','hpc':'HPC','amy':'AMY','acc':'ACC','ofc':'OFC','ains':'aINS',
#                   'pins':'pINS','dmpfc':'dmPFC','dlpfc':'dlPFC','vlpfc':'vlPFC'}


In [ ]:
sns.set_context("poster")
fig,ax = plt.subplots(figsize=(15, 7),dpi=500)

fig_save_dir = f'{base_dir}ephys_analysis/figs/hfa_regressions/'
os.makedirs(fig_save_dir,exist_ok=True)

sns.barplot(data = roi_bdi_plot_data,
            x = 'roi',
            y = 'raw_decisionCPE:bdi',
#             y='elec_bCPE',
            color = '#561C46',
            saturation=0.4,
            order=roi_order,
            errorbar=None) 


sns.stripplot(data = roi_bdi_plot_data,
            x = 'roi',
            y = 'raw_decisionCPE:bdi',
#               y='elec_bCPE',
            color = '#561C46',
            alpha=.3,
              dodge=True,
            order=roi_order,
            size = 8)

ax.axhline(y=0,color='#561C46',linewidth=1.5)

# plt.ylim([-0.011,0.01])
ax.set(xlabel=None)
plt.tick_params(length=0)
plt.ylabel(r'$HFA_{{{zpow}}} \sim cpe:bdi$')
# plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.3f}')) 
# plt.locator_params(axis='y', nbins=8)


sns.despine()
    
plt.savefig(f'{fig_save_dir}/HFA_CPEBDIINT{date}.pdf', format='pdf', metadata=None,
bbox_inches=None, pad_inches=0.1,dpi=500,
facecolor='auto', edgecolor='auto',
backend=None)

In [ ]:
palette=dict(Low="#3f2755", High="#125549")